# Random Forest playground 
## My goals are
- Explore RF with tranditional decision trees (kd-trees)
- Explore RF approach with RP-trees
- Explore RF with oblique splits (PCA, LDA, etc)
- Explore RF with preprocessings
 * Random rotation (Vempala)
 * Randomer forest
 * Output space random projections

Building of random forest (bagging technique) is adapted from tutorial: http://machinelearningmastery.com/implement-random-forest-scratch-python/
and part of the design of my spatial-tree class is inspired by "spatialtree" package:https://github.com/bmcfee/spatialtree
(note: I believe my design is better than that of Brian McFee, since it is more concise and flexible)

On sources of data compression:
 - Forest has three sources of data compression, which may or may not involve randomness: 
  * preprocessing such as PCA, random projection, random rotation, etc, which can be applied either for a single tree or for the entire forest and achieves
    - feature selection/extraction for the entire forest
      * This includes the theoretically justified randomly oriented kd-tree in [Vempala 12]
    - feature selection/extraction for a single tree
  * data subsampling for each tree, which adds randomness
  * feature selection/extraction at each tree node (before splitting direction and value are determined)

 - A generalized forest where methods to achieve compression vary at the node level in [Tomita, Maggioni, Vogelstein 16], where at each step a p by d projection matrix A is chosen; this subsumes
  * Forest-IC, Forest-RC in Breiman: sparsity constrained projection matrix sampling
  * Rotation forest: deterministic projection matrix via PCA
  * Randomer forest: sparse projection matrix via "very sparse random projections" 
  * RP-tree proposed in Dasgupta can also be viewed as a special case here, where the matrix A is a 1-D dense or sparse projection vector
(note: when d>1, after projection, the splitting rule must be able to pick a coordinate for splitting, so the first three approaches are more suitable for supervised tasks where coordinate-selection can be made via labels)

In [1]:
import urllib
from six.moves import cPickle as pickle
import random 
from csv import reader
from math import sqrt
from math import floor
import os
import numpy as np
import scipy
import scipy.sparse as sps
from sklearn import random_projection, cluster, decomposition
from pandas import DataFrame
from IPython.display import display

In [32]:
###---------- Utility functions
## computing spread of one-dim data
def spread_1D(data_1D):
    if len(data_1D)==0:
        return 0
    return np.amax(data_1D)-np.amin(data_1D)

## computing data diamter of a cell
def data_diameter(data):
    """
    Input data is assumed to be confined in the
    desired cell
    """
    dist, indx, indy = 0, 0, 0
    if data.shape[0] == 1:
        return dist, indx, indy
    for i in range(data.shape[0]):
        for j in range(i+1, data.shape[0]):
            dist_new = np.linalg.norm(data[i,:]-data[j,:])
            if dist_new > dist:
                dist, indx, indy = dist_new, i, j
    return dist, indx, indy

###---------- compressive projection matrix designs
def comp_projmat(data, **kwargs):
    """
    returns a projection matrix
    Warning: the projection matrix returned can be either dense or sparse
    """
    namelist = ['breiman','principal','tomita', 'twomeans_proj' ,'dasgupta']
    assert kwargs['name'] in namelist, "No such method for constructing projection matrix!"
    
    if kwargs['name'] == 'breiman':
        ## Breiman's Forest-IC and Forest-RC
        s = kwargs['sparsity']
        d = kwargs['target_dim']
        A = np.zeros((data.shape[1],d))
        ## sample sparsity-constrained A
        for i in range(d):
            ind = np.random.choice(data.shape[1], size=s, replace=False)
            if s == 1:
                ## Forest-IC
                A[ind,i] = 1
            else:
                for j in range(len(ind)):
                    A[ind[j], i] = np.random.uniform(-1,1)
    
    elif kwargs['name'] == 'principal':
        ## Project to principal components of the data
        A = np.zeros([data.shape[1],1])
        ## find A by PCA
        alg = decomposition.PCA(n_components=1)
        alg.fit(data)
        A[:,0] = alg.components_.T
        
    elif kwargs['name'] == 'tomita':
        ## randomer forest (sparse version of random forest)
        d = kwargs['target_dim']
        ## sample sparse A via very sparse rp
        density = 1/(data.shape[1]**(1/2)) #default density value
        if 'density' in kwargs:
            if kwargs['density'] <= 1 and kwargs['density']>0:
                density = kwargs['density']
        
        transformer = random_projection.SparseRandomProjection(n_components=d, density=density)    
        transformer.fit(data)
        A = transformer.components_.copy()
        A = A.T ## A is SPARSE!
    
    elif kwargs['name'] == 'twomeans_proj':
        ## project along the direction of two means
        d = kwargs['target_dim']
        A = np.zeros((data.shape[1],d))
        for i in range(d):
            ## sample columns of A by choosing pairs of means
            if 'params' in kwargs:
                if kwargs['params'] == 'D2':
                    ## D2-sampling (currently I'm using kmeans++ as a proxy)
                    sampler = cluster.KMeans(n_clusters=2, init='k-means++', n_init=1, max_iter=1)
                    sampler.fit(data)
                    C = sampler.cluster_centers_
                    A[:,i] = (C[0,:]-C[1,:]).T
                    break
            
            # default to Uniform sampling (without replacement)
            ind = np.random.choice(data.shape[0], size=2, replace=False)
            A[:,i] = (data[ind[0],:]-data[ind[1],:]).T
    else:
        ## dasgupta rp-tree 
        d = 1 # default to a random vector          
        if 'target_dim' in kwargs:
            d = kwargs['target_dim']
        n_features = data.shape[1]
        A = np.zeros((data.shape[1], d))
        # sample dense projection matrix
        for i in range(d):
            A[:,i] = np.random.normal(0, 1/np.sqrt(n_features), n_features)

    return A

#######-------split designs

## supervised splits
def cart_split(data, proj_mat, labels=None, regress=False):
    # test for the best split feature and threshold on data CART criterion
    if sps.issparse(proj_mat):
        #data_trans = sps.csr_matrix.dot(data, proj_mat).squeeze()
        data_trans = proj_mat.T.dot(data.T).T.squeeze()
    elif proj_mat is None:
        data_trans = data
    else:
        data_trans = np.dot(data, proj_mat) # n-by-d

    score = -999 
    ind = None 
    thres = None
    if data_trans.ndim == 1:
        if not regress:
            #classification
            score, thres = cscore(data_trans, labels)
        else:
            #regression
            score, thres = rscore(data_trans, labels)
        w = proj_mat
    
    else:
       
        for i in range(proj_mat.shape[1]):
            if not regress:
                # classification
                score_new, thres_new = cscore(data_trans[:,i], labels)
            else:
                # regression
                score_new, thres_new = rscore(data_trans[:,i], labels)
            if score_new > score:
                score = score_new
                ind = i
                thres = thres_new
        w = proj_mat[:,ind]
    return score, w, thres

def cscore(data_1D, labels):
    ## cart classification criterion
    score = -999
    thres = None
    if not list(labels):
        return score, thres
    n = len(labels)
    p1 = np.mean(labels)
    data_sorted, ind_sorted = np.sort(data_1D), np.argsort(data_1D)
    for i in range(1,n):
        cell_l = ind_sorted[:i]
        cell_r = ind_sorted[i:]
        split_val = data_sorted[i]
        if not list(labels[cell_l]) or not list(labels[cell_r]):
            #Do nothing if after either of the left and right labels are empty
            pass
        else:
            p1_l = np.mean(labels[cell_l])
            p1_r = np.mean(labels[cell_r])
            n_l = len(cell_l)
            score_new = p1*(1-p1) - (n_l/n)*(1-p1_l)*p1_l - (n-n_l)/n*(1-p1_r)*p1_r  
            if score_new > score:
                score = score_new
                thres = split_val
    return score, thres

def rscore(data_1D, labels):
    ## cart regression criterion
    score = -999
    thres = None
    if not list(labels):
        return score, thres
    n = len(labels)
    ybar = np.mean(labels)
    data_sorted, ind_sorted = np.sort(data_1D), np.argsort(data_1D)
    
    for i in range(1,n):
        cell_l = ind_sorted[:i]
        cell_r = ind_sorted[i:]
        split_val = data_sorted[i]
        if not list(labels[cell_l]) or not list(labels[cell_r]):
            #Do nothing if after either of the left and right labels are empty
            pass
        else:
            ybar_l = np.mean(labels[cell_l])
            ybar_r = np.mean(labels[cell_r])
            score_new =(np.sum((labels-ybar)**2)-np.sum((labels[cell_l]-ybar_l)**2)\
                        -np.sum((labels[cell_r]-ybar_r)**2))/n
            if score_new > score:
                score = score_new
                thres = split_val
    return score, thres

##### median splits
def median_spill_split(data, proj_mat, spill=0.25, labels=None):
    if sps.issparse(proj_mat):
        #data_transformed = sps.csr_matrix.dot(data, proj_mat).squeeze()
        data_trans = proj_mat.T.dot(data.T).T.squeeze()
    elif proj_mat is None:
        data_trans = data
    else:
        data_trans = np.dot(data, proj_mat) # n-by-d
    if data_trans.ndim > 1:
        score = 0
        ind = 0
        for i in range(proj_mat.shape[1]):
            score_new = spread_1D(data_trans[:,i])
            if score_new > score:
                score = score_new 
                ind = i
        w = proj_mat[:, ind]
        thres = scipy.stats.mstats.mquantiles(data_trans[:, ind], [0.5 - spill/2, 0.5 + spill/2])
    else:
        thres = scipy.stats.mstats.mquantiles(data_trans, [0.5 - spill/2, 0.5 + spill/2])
        w = proj_mat
        score = spread_1D(data_trans)
    return score, w, thres

def median_split(data, proj_mat, labels=None):
    if sps.issparse(proj_mat):
        #data_transformed = sps.csr_matrix.dot(data, proj_mat).squeeze()
        data_trans = proj_mat.T.dot(data.T).T.squeeze()
    elif proj_mat is None:
        data_trans = data
    else:
        data_trans = np.dot(data, proj_mat) # n-by-d
    if data_trans.ndim > 1:
        score = 0 
        ind = 0
        for i in range(proj_mat.shape[1]):
            score_new = spread_1D(data_trans[:,i])
            if score_new > score:
                score, ind = score_new, i
        w = proj_mat[:, ind]
        thres = np.median(data_trans[:,ind])
    else:
        thres = np.median(data_trans)
        w = proj_mat
        score = spread_1D(data_trans)
    return score, w, thres

def median_perturb_split(data, proj_mat, node_height, labels=None, diameter=None):

    if (node_height) % 2 == 0:
        # normal median splits
        return median_split(data, proj_mat, labels=labels)
    else:
        assert diameter is not None, "Diameter of the cell must be given!"
        # noisy splits
        if sps.issparse(proj_mat):
            #data_transformed = sps.csr_matrix.dot(data, proj_mat).squeeze()
            data_trans = proj_mat.T.dot(data.T).T.squeeze()
        elif proj_mat is None:
            data_trans = data
        else:
            data_trans = np.dot(data, proj_mat) # n-by-d
        
        if data_trans.ndim > 1:
            ## We use spread as the way to choose the best feature (could be changed)
            score, ind = 0, 0
            for i in range(proj_mat.shape[1]):
                score_new = spread_1D(data_trans[:, i])
                if score_new > score:
                    score, ind = score_new, i
            w = proj_mat[:, ind]
            jitter = np.random.uniform(-1,1) * 6/np.sqrt(data.shape[1]) * diameter
            thres = np.median(data_trans[:,ind])+jitter
        
        else:
            jitter = np.random.uniform(-1,1) * 6/np.sqrt(data.shape[1]) * diameter
            thres = np.median(data_trans)+jitter
            w = proj_mat
            score = spread_1D(data_trans)
        
    
        return score, w, thres

## cluster-based split
def cluster_based_split(data, proj_mat, sample_method='uniform', labels=None):
    # split at the bisector defined by two means
    if sps.issparse(proj_mat):
        #data_transformed = sps.csr_matrix.dot(data, proj_mat).squeeze()
        data_trans = proj_mat.T.dot(data.T).T.squeeze()
    elif proj_mat is None:
        data_trans = data
    else:
        data_trans = np.dot(data, proj_mat) # n-by-d
    
    if sample_method == 'uniform':
        ind = np.random.choice(data_trans.shape[0], size=2, replace=False)
        w = (data_trans[ind[0],:]-data_trans[ind[1],:]).T
        distance = np.linalg.norm(data_trans[ind[0],:]-data_trans[ind[1],:])
        x1 = data_trans[ind[0],:]
        
    elif sample_method == 'D2':
        sampler = cluster.KMeans(n_clusters=2, init='k-means++', n_init=1, max_iter=1)
        sampler.fit(data)
        C = sampler.cluster_centers_
        w = (C[0,:]-C[1,:]).T
        distance = np.linalg.norm(C[0,:]-C[1,:])
        x1 = C[0,:]

    else:
        print("Unrecognized cluster-based splitting method!")
    
    w = w/np.linalg.norm(w)
    thres = np.dot(x1, w)+distance/2
    score = spread_1D(np.dot(data_trans, w))
    
    return score, w, thres

#######-------stop rules
def naive_stop_rule(data, height=None):
    if data.shape[0] <= 1:
        return True
    if height > 8:
        ## DO NOT ever make it exceed 15!!!
        return True
    
    return False

def naive_DT_stop_rule(data, labels, height=None):
    if data.shape[0] <= 1:
        return True
    if height > 8:
        return True
    if np.mean(labels)==0 or np.mean(labels)==1:
        return True

def cell_size_rule(data, height, max_height=None, target_diameter=None):
    ddiameter,_,_ = data_diameter(data)
    if target_diameter < 0.01:
        return True
    if ddiameter <= target_diameter:
        return True
    elif max_height is not None and height > max_height:
        return True
    else:
        return False
    

In [33]:
## A class of binary spatial-trees 
        
class flex_binary_trees(object):
    """
    A recursive data structure based on binary trees
     - at each node, it contains data, left, right child (or none if leaf), just as any other binary tree
     - it also knows its height
     - additionally, it has meta information about split direction and split threshold
     - to incorporate the use of master-slave trees (see below), it also has an optional reference
      to a master tree
     
    On splitting method
     - if rpart or cpart are used, labels must be provided
    """

    def __init__(self, data, data_indices=None, 
                 proj_design={'name':'projmat','params':{'name':'breiman','sparsity':3,'target_dim':10}}, 
                 split_design={'name':'cart', 'params':{'regress':False}}, 
                 stop_design={'name':'naive'}, 
                 height=0, labels=None, predict_type=None, master_tree=None):
        """
        data: n by d matrix, the entire dataset assigned to the tree
        data_indices: the subset of indices assigned to this node
        proj_design: A dictionary that contains name and params of a method; 
          the method returns one or more splitting directions (projection matrix)
        split_design: A dict that contains name and params of a function;
          the function s.t. given 1D projected data, it must return the splitting threshold
        stop_rule: a boolean function of data_indices and height
        height: height of current node (root has 0 height)
        """
        assert data_indices is not None, "You must pass data indices to root!"
        self.data = data
        self.data_ind = data_indices.copy()
        ## This default assignment will cause problems in recursion!
        #if data_indices is None:
        #    self.data_ind = np.ones(data.shape[0], dtype=bool)
        self.proj_design = proj_design
        self.split_design = split_design 
        self.stop_design = stop_design
        self.predict_type = predict_type
        
        self.height_ = height
        self.labels = labels
        self.master_tree = master_tree
        self.leftChild_ = None
        self.rightChild_ = None
            
            
        ## set stop rule: a boolean function
        if np.sum(self.data_ind) == 0:
            ## stop if this cell has no data
            self.isLeaf = True
        elif self.stop_design['name'] == 'naive':
            self.isLeaf = naive_stop_rule(self.data[self.data_ind,:], height=self.height_) 
            
        elif self.stop_design['name'] == 'cell_size':
            assert 'params' in self.stop_design, "Please specify stopping parameters!"
            d0 = self.stop_design['params']['diameter']
            max_h = None
            if 'max_level' in self.stop_design['params']:
                max_h = self.stop_design['params']['max_level']
            self.isLeaf = cell_size_rule(self.data[self.data_ind,:], self.height_,
                                         max_height=max_h, target_diameter=0.5*d0)
        else:
            print("You must provide a known stopping method!")
            self.isLeaf = True
            
    
    def proj_rule_function(self):
        """
        A function such that given name of a method, returns a projection vector (splitting direction)
        Can be override (user-defined)
        returns a n_features by n_projected_dim projection matrix, A
        
        Warning: Should only be executed if self.data_ind has at least ONE nonzero element
        """
        assert np.sum(self.data_ind) > 0, "The cell is empty!!"
        name_list = ['projmat', 'cyclic', 'full']
        
        method = self.proj_design['name']
        
        assert method in name_list, 'No such rule implemented in the current tree class!'
        
        if method == 'projmat':
            
            return comp_projmat(self.data[self.data_ind,:], **self.proj_design['params'])
        
        elif method == 'cyclic':
            # cycle through features using height information
            # here A is 1D
            n_features = self.data.shape[1]
            A = np.zeros(n_features)
            A[self.height_ % n_features] = 1
            return A
        else:
            # no compression, 'full'
            return np.eye(self.data.shape[1])
        
    def split_rule_function(self, A):
        """
        Given a projection matrix
        Returns the best split direction and threshold
        Warning: Should only be executed if self.data_ind has at least ONE nonzero element
        """
        assert np.sum(self.data_ind) > 0, "The cell is empty!"
        name_list = ['cart', 'median', 'median_perturb', 'median_spill', 'cluster_based']
        
        method = self.split_design['name']
        assert method in name_list, 'No such split rule implemented in current tree class!'
        
        if 'params' in self.split_design:
            params = self.split_design['params']
        else:
            params = dict()
        
        if method == 'cart':
            return cart_split(self.data[self.data_ind,:], A, self.labels[self.data_ind], **params)
        elif method == 'median':
            return median_split(self.data[self.data_ind,:], A, **params)
        elif method == 'median_perturb':
            
            node_height = self.height_ # height of this node relative to root of the flex tree
            return median_perturb_split(self.data[self.data_ind,:], A, node_height, **params)
        elif method == 'median_spill':
            return median_spill_split(self.data[self.data_ind,:], A, **params)
        else:
            ## method == 'cluster_based'
            return cluster_based_split(self.data[self.data_ind,:], A, **params)
        
    
    def buildtree(self):
        """
        Recursively build a tree starting from current node as root
        Constructs w (projection direction) and threshold for each node
        
        To execute buildtree, self.data_ind must have at least ONE non-zero entry
        """
        if self.split_design['name'] == 'cart':
            assert self.labels is not None, "You must provide data labels to execute CART!"
        if not self.isLeaf:
            ## set projection/transformation/selection matrix
            A = self.proj_rule_function()  # A can be dense or sparse matrix
    
            ## find the best split feature and the best threshold
            split_rule = self.split_design['name']
            _, self.w_, self.thres_ = self.split_rule_function(A)
            
            ## transform data to get one or more candidate features
            if sps.issparse(self.w_):
                projected_data = sps.csr_matrix.dot(self.data[self.data_ind, :], self.w_).squeeze()
            else:
                projected_data = np.dot(self.data[self.data_ind, :], self.w_) ## project data to 1-D
            
            data_indices = []
            ## data_ind always has the same size as the number of data size
            ## data_indices has the same size as the number of data in this cell
            for i in range(len(self.data_ind)):
                if self.data_ind[i] == 1:
                    data_indices.append(i)
            assert len(data_indices) == len(projected_data)
            data_indices = np.array(data_indices)
            
            ## split data into left and right
            if isinstance(self.thres_, float):
                left_indices = projected_data < self.thres_
                right_indices = projected_data >= self.thres_
            elif isinstance(self.thres_, np.ndarray):
                left_indices = projected_data < self.thres_[1]
                right_indices = projected_data > self.thres_[0]
            else:
                print('Err: soemthing is wrong with the splitting threshold')
                exit(1)
            
            ## Here, it's still possible that one of the left or right indices is empty array
            #display(left_indices)
            #display(right_indices)
            #display(data_indices)
            if self.split_design['name'] != 'median_spill':
                assert np.sum(left_indices)+np.sum(right_indices) == len(data_indices)
            left_ind = data_indices[left_indices]
            right_ind = data_indices[right_indices]
            ##
            n_data = self.data.shape[0]
            left = np.zeros(n_data, dtype=bool)
            if list(left_ind):
                # make assingment only if left_ind is non-empty
                left[left_ind] = 1
            right = np.zeros(n_data, dtype=bool)
            if list(right_ind):
                right[right_ind] = 1
            
            ## build subtrees on left and right partitions
            ## By our choice, empty cell will still make a node
            self.leftChild_ = flex_binary_trees(self.data, left, self.proj_design, 
                                                    self.split_design, self.stop_design, 
                                                    self.height_+1, self.labels)
            self.leftChild_.buildtree()
            
            self.rightChild_ = flex_binary_trees(self.data, right, self.proj_design, 
                                                     self.split_design, self.stop_design, 
                                                     self.height_+1, self.labels)
            self.rightChild_.buildtree()
            
        
    def train(self):
        self.buildtree()
        
    def predict_one(self, point):
        return predict_one_bt(self, point, predict_type=self.predict_type)
        
    def predict(self, test):
        return predict_bt(self, test, predict_type=self.predict_type)
            
            
####-------- Utility functions for binary trees   

def retrievalLeaf(btree, query):
    """
    Given a binary partition tree
    returns a node that contains query
    Here, the cell of a returned leaf node may be empty
    """
    if btree.leftChild_ is None and btree.rightChild_ is None:
        return btree
    if sps.issparse(btree.w_):
        val = sps.csr_matrix.dot(btree.w_.T, query).squeeze()
    else:
        val = np.dot(btree.w_, query)
    if val < btree.thres_:
        return retrievalLeaf(btree.leftChild_, query)
    else:
        return retrievalLeaf(btree.rightChild_, query)
    
def retrievalSet(btree, query):
    """
    Given a binary partition tree
    returns indices of points in the cell that contains query point
    By design, this set of indices will NOT be all zeros
    """
    ## base case: return data indices if leaf node is reached    
    if btree.leftChild_ is None and btree.rightChild_ is None:
        ## By our recursion, data_ind returned won't be all zeros
        assert np.sum(btree.data_ind) != 0, "Something is wrong!"
        return btree.data_ind
        
    ## check which subset does the query belong to
    if sps.issparse(btree.w_):
        val = sps.csr_matrix.dot(btree.w_.T, query)
    else:
        val = np.dot(btree.w_, query)
        
    if isinstance(btree.thres_, float):
        if val < btree.thres_: 
            if (btree.leftChild_ is None) or (np.sum(btree.leftChild_.data_ind)==0):
                # use parent cell as the retrieval set 
                # parent cell is guaranteed to be non-empty
                return btree.data_ind
            return retrievalSet(btree.leftChild_, query)
        else:
            if (btree.rightChild_ is None) or (np.sum(btree.rightChild_.data_ind)==0):
                return btree.data_ind
            return retrievalSet(btree.rightChild_, query)
    elif isinstance(btree.thres_, np.ndarray):
        S = set()
        if val < btree.thres_[1]:
            if (btree.leftChild_ is None) or (np.sum(btree.leftChild_.data_ind)==0):
                # use parent cell as the retrieval set 
                # parent cell is guaranteed to be non-empty
                S |= btree.data_ind
            else: 
                S |= retrievalSet(btree.leftChild_, query)
        if val > btree.thres_[0]:
            if (btree.rightChild_ is None) or (np.sum(btree.rightChild_.data_ind)==0):
                # use parent cell as the retrieval set 
                # parent cell is guaranteed to be non-empty
                S |= btree.data_ind
            else: 
                S |= retrievalSet(btree.rightChild_, query)
        return S
            
    else:
        print('Err: soemthing is wrong with the splitting threshold')
        exit(1)
        
def getDepth(btree, depth):
    """
    find out the depth (maximal height of all branch) of a binary tree
    depth is the depth of current node
    """
    ## via DFS
    if btree.leftChild_ is None and btree.rightChild_ is None:
        return depth
        
    depthL = getDepth(btree.leftChild_, depth+1)
    depthR = getDepth(btree.rightChild_, depth+1)
        
    if depthL >= depthR:
        return depthL
    else:
        return depthR
        
def predict_one_bt(btree, point, predict_type='class'):
    assert list(btree.labels), "This tree has no associated data labels!"
    set_ind = retrievalSet(btree, point) 
    
    if predict_type == 'class':
        if np.sum(set_ind)==0:
            # using retrievalSet makes sure set_ind is not all zeros
            # but this check is just in case
            return round(np.mean(btree.labels))
        return round(np.mean(btree.labels[set_ind]))
        
    else:
        # regression
        if np.sum(set_ind)==0:
            return np.mean(btree.labels)
        return np.mean(btree.labels[set_ind])
        
    
def predict_bt(btree, test, predict_type='class'):
    """
    Returns a list of predictions corresponding to test set
    """
    predictions = list()
    for point in test:
        predictions.append(predict_one_bt(btree, point, predict_type=predict_type))
    return predictions

        
def k_nearest(tree, query):
    """
    Given dataset organized in a tree structure  ----- TO BE IMPLEMENTED
    find the approximate k nearest neighbors of query point
    """
    return k_nearest
        
    
###
def printPartition(tree, level):
    """
    Starting from root of the tree, traverse each node at given level
    and print the partitioning it holds
    Can be used for testing purposes
    """
    if tree.height_ == level or (tree.leftChild_ is None and tree.rightChild_ is None):
        ind_set = []
        for i in range(len(tree.data_ind)):
            if tree.data_ind[i] == 1:
                ind_set.append(i)
        print(ind_set)
    else:
        printPartition(tree.leftChild_, level)
        printPartition(tree.rightChild_, level)
        
def traverseLeaves(tree):
    """
    This traversal works for both balanced and unbalanced binary trees
    """
    if tree.leftChild_ is None and tree.rightChild_ is None:
        # leaf node
        yield tree
    
    if tree.leftChild_ is not None:
        for t in traverseLeaves(tree.leftChild_):
            yield t
    if tree.rightChild_ is not None:
        for t in traverseLeaves(tree.rightChild_):
            yield t
        


In [4]:
#######------- A class of master trees inspired by Kpotufe's adaptive tree structure
## these are not binary trees but are "meta-trees" built on binary trees
## it is used to prune a binary tree as on-the-fly

class master_trees(object):
    
    def __init__(self, data, labels=None, parent_slave_tree=None, child_slave_tree_params=None, 
                 curr_height=0, max_height=10):
        """
        Each instance of this class has two links: a link to a parent slave tree and a link to
        a child slave tree; each slave tree is an instance of the flex_binary_tree class
        Exceptions:
            -Leaf nodes only have parent slave trees
            -Root node only has a child slave tree
        
        data: data of the entire dataset
        (unlike flex_binary_tree, each node of the master tree doesn't have data_indices as attribute; it is because
        this information is already contained in its slave tree;
        in fact, there is no need to use "data" as attribute either?)
        parent_slave_tree: slave_tree that leads to the creation of this master_tree 
            - if None, master tree is root 
            - the parent slave tree also has a pointer to this master tree
        child_slave_tree_params: a dict of parameters of the slave tree if default (RP-tree) is not used
            - if not None, it should has keys "proj_design", "split_design", "stop_design"
        child_slave_tree: slave tree that is constructed by this master tree, by calling grow_child
            - the child slave tree also has a pointer to this master tree(is this necessary?)
        curr_height: absolute level of this master tree node, counting internal levels in slave trees
        max_height: maximal level that is allowed to be reached by the entire tree 
            - used independent of the slave tree stopping test
        """
        self.data = data
        self.labels = labels
        self.slave_tree_params=child_slave_tree_params
        self.n_rep = 0
        if self.slave_tree_params is not None and 'repeat' in self.slave_tree_params:
            self.n_rep = self.slave_tree_params['repeat']
        self.curr_height = curr_height
        self.max_height = max_height
        self.leaves_list = list()
        self.parent_slave_tree = None
        
        ## initialize parent slave tree if exists
        if parent_slave_tree is not None:
            self.parent_slave_tree = parent_slave_tree
            self.parent_slave_tree.master_tree = self
        ## for testing purpose
        print_diam_leaves(self)
        
    def grow_child(self):
        """
        Method to grow a child slave tree from root to leaves
        """ 
        if self.parent_slave_tree is None and self.slave_tree_params is None:
            # default parameters
            data_ind = np.ones(self.data.shape[0], dtype=bool)
            proj_design = {'name':'projmat', 'params':{'name':'dasgupta'}}
                
            ## Here, data diameter is the diameter of the entire dataset (since this is the root of master tree)
            ddiameter,_,_ = data_diameter(self.data) 
            #print(self.data.shape)
            split_design = {'name':'median_perturb', 'params':{'diameter':ddiameter}}
            stop_design = {'name':'cell_size'}
            stop_design['params']={'diameter':ddiameter}
            #print("pass")
            
        elif self.parent_slave_tree is None and self.slave_tree_params is not None:
            # user-defined child_slave_tree params
            data_ind = np.ones(self.data.shape[0], dtype=bool)
            proj_design = self.slave_tree_params['proj_design']
            split_design = self.slave_tree_params['split_design']
            stop_design = self.slave_tree_params['stop_design']
        elif self.parent_slave_tree is not None:
            # this will be invoked as long as this master tree is not a root
            # if a parent slave tree exists, 
            # we always use its params to define the new child slave tree
            data_ind = self.parent_slave_tree.data_ind
            proj_design = self.parent_slave_tree.proj_design
            stop_design = self.parent_slave_tree.stop_design
            #print("Current diameter stored is %f" %stop_design['params']['diameter'])
            split_design = self.parent_slave_tree.split_design
            stop_design = self.parent_slave_tree.stop_design
        else:
            print("Something is wrong")
        
        ## create child slave tree
        # By design, we use the relative height in child slave tree
        self.child_slave_tree = flex_binary_trees(self.data, data_indices=data_ind, proj_design=proj_design,
                                        split_design=split_design, stop_design=stop_design,
                                        height=0, labels=self.labels, master_tree=self)
        ### employs child slave tree      
        self.child_slave_tree.buildtree()
        depth = getDepth(self.child_slave_tree, 0) #get relative depth of the child slave tree
        print("The relative depth of the new child tree is %d" %depth)
        for i in range(self.n_rep):
            ## select tree with shortest depth
            slave_tree = flex_binary_trees(self.data, data_ind, proj_design=proj_design, split_design=split_design, 
                              stop_design=stop_design, height=0, labels=self.labels, master_tree=self)
            slave_tree.buildtree()
            if depth > getDepth(slave_tree, 0):
                self.child_slave_tree = slave_tree
        
        
    def iter_child_leaves(self):
        return traverseLeaves(self.child_slave_tree)
        
    def build_master_trees(self):
        self.grow_child()
        leaves_gen = self.iter_child_leaves() # up to this point, nodes are binary trees
        
        for leaf in leaves_gen:
            ## by design, curr_height is the absolute height of this master node (counting slave tree levels)
            # Update the diameter of data in leaf
            ddiam,_,_ = data_diameter(self.data[leaf.data_ind])
            if 'params' in leaf.split_design and 'diameter' in leaf.split_design['params']:
                leaf.split_design['params']['diameter'] = ddiam
            if 'params' in leaf.stop_design and 'diameter' in leaf.stop_design['params']:
                leaf.stop_design['params']['diameter'] = ddiam
            
            # child tree leaves will become parent slave trees for the new master tree
            new_master_tree = master_trees(self.data, labels=self.labels, parent_slave_tree=leaf, 
                                            curr_height=leaf.height_+self.curr_height ,max_height=self.max_height )
            #display(new_master_tree.parent_slave_tree)
            self.leaves_list.append(new_master_tree)
            
        ## Decide whether to grow the next level of master trees
        if not mtree_test_stop(self.leaves_list, self.max_height):
            for mleaf in self.leaves_list:
                if np.sum(mleaf.parent_slave_tree.data_ind) > 1:
                    ## if max height is not achieved in any cell of slave tree
                    # and that the cell is not empty, continue grow it
                    print("A new master tree built!")
                    mleaf.build_master_trees()
                
    def train(self):
        ## interface with cross-validation evalutaion
        self.build_master_trees()
        
    def predict_one(self, point, predict_type='class'):
        return predict_one_mt(self, point, predict_type=predict_type)
     
    def predict(self, test, predict_type='class'):
        ## interface with cross-validation evaluation
        return predict_mt(self, test, predict_type=predict_type)

            

####----------- Utility functions for master trees
def mtree_test_stop(mtree_leaves_list, max_height):
    count=1
    for mtree_leaf in mtree_leaves_list:
        if mtree_leaf.curr_height >= max_height:
            #print("Stop test is passed at height %d" %mtree_leaf.curr_height)
            return True
        print("Depth of the %d-th leaf is %d" %(count,mtree_leaf.curr_height))
        data_ind = mtree_leaf.parent_slave_tree.data_ind
        ddiam,_,_ = data_diameter(mtree_leaf.data[data_ind,:])
        #print("Diameter of the %d-th leaf is %f" %(count, ddiam))
        count+=1
    
    return False
       
def retrievalLeaf_mtree(mtree, query):
    ## base case
    if not mtree.leaves_list:
        return mtree
    
    ##
    #find the leaf containing query using its binary slave tree
    slave_leaf = retrievalLeaf(mtree.child_slave_tree, query) 
    return retrievalLeaf_mtree(slave_leaf.master_tree, query) #recurse on master tree of the found leaf

def traverseLeaves_mtree(mtree):
    if not mtree.leaves_list:
        yield mtree
    
    for leaf in mtree.iter_child_leaves():
        traverseLeaves_mtree(leaf.master_tree)


def print_mtree_leaves(mtree):
    count = 0
    if not mtree.leaves_list:
        if mtree.parent_slave_tree is not None:
            indices = np.arange(mtree.data.shape[0])
            display(indices[mtree.parent_slave_tree.data_ind])
            #return np.sum(mtree.parent_slave_tree.data_ind)
            return count+1
        else:
            print("Root node")
            
    else:
        for leaf in mtree.leaves_list:
            count += print_mtree_leaves(leaf)
        return count

def apply_mtree_leaves(mtree, myfunc):
    if not mtree.leaves_list:
        if mtree.parent_slave_tree is not None:
            myfunc(mtree.parent_slave_tree)
        else:
            print("Root node")
    else:
        for leaf in mtree.leaves_list:
            apply_mtree_leaves(leaf)

def print_diam_leaves(mtree):
    def myfunc(f_tree):
        ddiam,_,_ = data_diameter(f_tree.data[f_tree.data_ind])
        print(ddiam)
    apply_mtree_leaves(mtree, myfunc)
    
    
def predict_one_mt(mtree, point, predict_type='class'):
    """
    In the future, change retrieval method to retrievalSet_mtree, which should have 
    better performance
    """
    mLeaf = retrievalLeaf_mtree(mtree, point)
    set_ind = mLeaf.parent_slave_tree.data_ind
    
    if predict_type == 'class':
        #print(round(np.mean(mcell.slave_tree.labels[set_ind])))
        if np.sum(set_ind)==0:
            # if set index is an empty list
            return round(np.mean(mLeaf.parent_slave_tree.labels))
        return round(np.mean(mLeaf.parent_slave_tree.labels[set_ind]))
        
    else:
        # regression
        if np.sum(set_ind)==0:
            return np.mean(mLeaf.parent_slave_tree.labels)
        return np.mean(mLeaf.parent_slave_tree.labels[set_ind])
    
def predict_mt(mtree, test, predict_type='class'):
    """
    Returns a list of predictions corresponding to test set
    """
    predictions = list()
    for point in test:
        predictions.append(predict_one_mt(mtree, point, predict_type=predict_type))
    return predictions        
            

In [5]:
class forest(object):
    def __init__(self, data, labels=None, tree_design=None, predictor_type='class',
                  n_trees=10, n_samples=100, n_features=None):
        """
        tree_design: A dictionary containing 
          - tree: specifies type of the tree; flex or master
          - a proj_design dictionary, 
          - a split_design dictionary,
          - a stop_design dictionary
        
        """
        self.data = data
        self.labels = labels
        self.tree_design = tree_design
        self.predictor_type = predictor_type
        self.n_trees = n_trees
        self.trees = list() # store trees for re-use
        self.n_samples = n_samples
        self.n_features = n_features
        
        
    def reset_sample_size(self, n_samples=None, n_features=None):
        if n_samples is not None:
            self.n_samples = n_samples
        if n_features is not None:
            self.n_features = n_features
            
    def reset_predictor_type(self, method):
        self.predictor_type = method
        
    def build_forest_classic(self, isPredict=True):
        if isPredict:
            assert labels is not None, "Data labels missing!"
        tree_type = self.tree_design['tree']
        
        for i in range(self.n_trees):
            # sample data points with replacement
            # note numpy indexing supports repetitive/duplicate indexing
            data_ind = np.random.choice(self.data.shape[0], self.n_samples, replace=True)
            data_tree = self.data[data_ind,:] # data unique to this tree
            
            if self.n_features is not None:
                ## optionally subsample features (note: this is NOT done in the original RF)
                feature_ind = np.random.choice(self.data.shape[1], self.n_features, replace=True)
                data_tree = data_tree[:,feature_ind] # features unique to this tree
            
            if isPredict:
                labels_tree = self.labels[data_ind]
            else:
                labels_tree = None
            
            if tree_type == 'flex':
                proj_design = self.tree_design['proj_design']
                split_design = self.tree_design['split_design']
                stop_design = self.tree_design['stop_design']
                    
                f_tree = flex_binary_trees(data_tree, np.ones(data_tree.shape[0], dtype=bool), 
                                        proj_design,split_design, stop_design, labels=labels_tree)
                f_tree.buildtree()
                self.trees.append(f_tree)
            
            elif tree_type == 'master':
                ## adaptive tree
                if self.tree_design is not None:
                    # user-defined adaptive tree
                    m_tree = master_trees(data_tree, labels=labels_tree, 
                                                   child_slave_tree_params = self.tree_design)
                else:
                    # default use Kpotufe's adaptive tree
                    m_tree = master_trees(data_tree, labels=labels_tree)
                
                m_tree.build_master_trees()
                self.trees.append(m_tree)
            
            else:
                ## scikit implementation of DC tree
                pass
    
    
    def build_forest_with_tree_preproc(self, method):
        pass
        
    def build_forest_with_forest_preproc(self, method):
        pass
    
    
    def train(self):
        self.build_forest_classic(isPredict=True)
    
    def predict_one(self, point):
        """
        Predictor_type can be either 'class' for classification,
        'regress' for regression, or a user-defined callable function
        """
        assert self.trees, "You must first build a forest"
        
        if self.predictor_type == 'class':
            ## binary classification
            avg_predict = 0
            for tree in self.trees:
                avg_predict += tree.predict_one(point, predict_type='class')
            if avg_predict/float(len(self.trees)) > 0.5:
                return 1
            else:
                return 0
        elif self.predictor_type == 'regress':
            ## regression
            avg_predict = 0
            for tree in self.trees:
                avg_predict += tree.predict_one(point, predict_type='regress')
            return avg_predict/float(len(self.trees))
        else:
            print("Unrecognized prediction method!")
           
    def predict(self, test):
        predictions = list()
        for point in test:
            predictions.append(self.predict_one(point))
        return predictions

In [6]:
## Utility functions for sampling and evaluation

def subsample(data, n_samples, n_features=None):
    """
    sample WITH replacement
    NOTE: This method is currently UNUSED
    """
    ind_data = np.random.choice(data.shape[0], size=n_samples, replace=True)
    if n_features is not None:
        ind_features = np.random.choice(data.shape[1], size=n_features, replace=True)
        return data[ind_data, ind_features]
        
    return data[ind_data,:] 

def cross_valid_split(n_data, n_folds):
    """
    Given size of the data and number of folds
    Returns n_folds disjoint sets of indices, where indices
    in each fold are chosen u.a.r. without replacement
    """
    data_ind = list(range(n_data))    
    folds = list()
    fold_size = floor(n_data/n_folds)
    for i in range(n_folds):
        if i < n_folds-1:
            fold = list()
            while len(fold) <= fold_size:
                index = random.randrange(len(data_ind))
                fold.append(data_ind.pop(index))
            folds.append(fold)
        else:
            ## assign all remaining data to the last fold
            folds.append(data_ind)
    return folds

def zero_one_loss(labels, predictions):
    correct = 0
    for i in range(len(labels)):
        if labels[i] == predictions[i]:
            correct += 1
    loss = (len(labels)-correct)/float(len(labels))
    return loss

def explained_var_loss(labels, predictions):
    #res_var = np.sum(np.array([diff**2 for diff in labels-predictions]))
    res_var = np.var(np.array(labels)-np.array(predictions))
    tot_var = np.var(np.array(labels))
    
    return 1-res_var/tot_var

def l2_loss(labels, predictions):
    loss = np.linalg.norm(np.array(labels)-np.array(predictions))
    return loss/(len(labels)**(1/2))

def csize_decrease_rate(data, tree):
    """
    This is an unsupervised evaluation, which tries to capture how fast
    the data size of a cell decreases after building a tree
    """
    diam_s,_,_ = data_diameter(data)
    diam_f = 0
    
    if hasattr(tree, 'slave_tree'):
        ## if this is a master tree
        for leaf in traverseLeaves_mtree(tree):
            diam, _, _ = data_diameter(leaf.slave_tree.data[leaf.slave_tree.data_ind, :])
            if diam > diam_f:
                diam_f = diam
            
    else:
        ## if this is normal binary tree
        for leaf in traverseLeaves(tree):
            diam, _, _ = data_diameter(leaf.data[leaf.data_ind,:])
            if diam > diam_f:
                diam_f = diam
        
    return (diam_s/diam_f)/getDepth(tree)
        

def cross_valid_eval(data, labels, n_folds, loss, algorithm, need_ind=False, **kwargs):
    """
    Given data and labels, a loss function, and a method
    generate a list of cv-losses
    
    """
    ## generate random folds
    folds_ind = cross_valid_split(data.shape[0], n_folds)
    losses = list()
    for fold_ind in folds_ind:
        print("Evaluating the %d-th fold" %(len(losses)+1))
        test_ind = fold_ind
        folds_ind_ = list(folds_ind) # this ensures we are not modifying the original list!
        folds_ind_.remove(fold_ind)
        train_ind = [item for sublist in folds_ind_ for item in sublist] #flatten remaining index set
        ## Further divide the data into train and test
        data_tr = data[train_ind,:]
        labels_tr = labels[train_ind]
        data_tt = data[test_ind,:]
        labels_tt = labels[test_ind]
        # train the algorithm 
        if need_ind:
            data_ind = np.ones(data_tr.shape[0], dtype=bool)
            alg = algorithm(data_tr, data_indices=data_ind, labels=labels_tr, **kwargs) #init
        else:
            # RF and master trees don't need to be given an index 
            alg = algorithm(data_tr, labels=labels_tr, **kwargs) #init
        alg.train()
        #c = print_mtree_leaves(alg) ## added should be removed
        #print("There are %d partitions" %c)
        
        # calculate loss on the current fold
        losses.append(loss(labels_tt, alg.predict(data_tt)))
        #print(labels[0],alg.predict(data_tt)[0])
        del alg
    return losses                   

In [7]:
######--- Utility functions for data preprocessing

# get data from url link and store
def download_data(fname, url_name, force = False):
    if force or not os.path.exists(fname):
        print('Downloading data from the internet...')
        try:
            urllib.urlretrieve(url_name, fname)
        except Exception as e:
            print("Unable to retrieve file from given url")
    else:
        print("File already exists")
        

# pickle or get pickled data with desired dataname
def maybe_pickle(dataname, data = None, force = False, verbose = True):
    """
    Process and pickle a dataset if not present
    """
    filename = dataname + '.pickle'
    if force or not os.path.exists(filename):
        # pickle the dataset
        print('Pickling data to file %s' % filename)
        try:
            with open(filename, 'wb') as f:
                pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save to', filename, ':', e) 
    else:
        print('%s already present - Skipping pickling.' % filename)
        with open(filename, 'rb') as f:
            data = pickle.load(f)

    return data


In [8]:
## Download sonar dataset for classification problems
fname = "sonar.all_data.csv"
url_name = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
download_data(fname, url_name)

File already exists


In [9]:
#### module for preprocessing sonar data and pickling it

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
# Convert string column to float
# this is for data conversion
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
 
# Convert string
# this is for label conversion
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup


### load and preprocess data
fname = 'sonar.all_data.csv'
data = load_csv(fname)
for i in range(len(data[0])-1):
    str_column_to_float(data, i)
    
lookup = str_column_to_int(data, -1)
data = np.array(data)
## random shuffling
pInd = np.random.permutation(data.shape[0])
display(pInd)
data = data[pInd,:]
X = data[:,:data.shape[-1]-1]
labels = data[:, -1]
print("Preprocessed data has shape %d by %d" % X.shape)
print("Preprocessed labels has length %d" % len(labels))
## pickle data
data_dict=maybe_pickle('sonar.all_data', {'data':X, 'labels':labels},force=True) 

array([  8,  75,  59, 197,  51, 122, 192, 113, 184, 140, 138,  88, 120,
        31, 141, 159, 171,   6, 100,  81, 189,  72,  64,  49, 109,  80,
       190,  69, 185,  18, 121, 158, 151,  30,  48, 157,   1, 103, 128,
        74, 111, 191, 118,  78,  37, 139, 194,  40,  63, 160,   7, 181,
        98, 108, 155, 168,  94, 137, 150, 146,  53,  36,  96,  62, 199,
       116, 186, 154,  44, 117, 162,  73, 169,  46,  24, 148,  95, 123,
       142, 132,  38, 102, 125,  92, 101, 112,  67, 135,  26,  86,  71,
        41,  91, 136, 207, 152, 124,  20,   2,  65,   9,  10,  68, 161,
       165,  19,   5,  89,  14,  47,  83,  43,  12, 193, 183,  34, 206,
         3, 196,  45,  50,  16, 201,  55, 105,  13,  79,  56, 167, 174,
       163, 133,  60,  77,  21, 164, 176,  39,  76, 173, 182,  29, 170,
       198,  90, 107, 126, 200,  84,  70, 179,  42, 134, 203, 130, 180,
       114,  66, 110,  23,  33, 172,  27, 175,  52,  22,  25, 143, 187,
       202, 144, 119,   4,  32, 178,  17, 177, 205, 129,  11,  3

Preprocessed data has shape 208 by 60
Preprocessed labels has length 208
Pickling data to file sonar.all_data.pickle


In [10]:
data = data_dict['data']
labels = data_dict['labels']
data_tr = data[:104,:]
labels_tr = labels[:104]
data_tt = data[104:,:]
labels_tt = labels[104:]
print("number of one's in labels are %d" %labels.sum())
print("number of one's in training labels are %d" %labels_tr.sum())

number of one's in labels are 111
number of one's in training labels are 57


In [ ]:
### Test: CART-decision tree + Breiman's features selection/projection rule
sparsity = [1, 3, 10]
target_dim = [1, 5, 10]
stop_rule = ['naive', 'cell_size']
scores_c1 = list()
scores_c1_test = list()
params_c1 = list()

##
for s in sparsity:
    for t_dim in target_dim:
        for s_rule in stop_rule:
            proj_design={'name':'projmat','params':{'name':'breiman','sparsity':s,'target_dim':t_dim}}
            split_design={'name':'cart'} 
            stop_design={'name':s_rule}
            if s_rule == 'cell_size':
                ddiam, _, _ = data_diameter(data_tr)
                stop_design['params'] = {'diameter':ddiam, 'max_level':2*np.log(data_tr.shape[0])}

            kwargs = {'proj_design':proj_design, 'split_design':split_design, 'stop_design':stop_design}
            scores_c1.append(cross_valid_eval(data_tr, labels_tr, 5, zero_one_loss, flex_binary_trees, 
                                              need_ind=True, **kwargs))
            scores_c1_test.append(cross_valid_eval(data_tt, labels_tt, 5, zero_one_loss, flex_binary_trees, 
                                              need_ind=True, **kwargs))
            params_c1.append([s, t_dim, s_rule])
            
#scores = cross_valid_eval(data_tr, labels_tr, 5, zero_one_loss, flex_binary_trees)
#score1 = np.mean(scores)

## best param: 3, 10, cell_size

In [ ]:
s_dict = {'sparsity':list(), 'target_dim':list(), 'stop_rule': list(), 'zero-one-loss':list(), 'zero-one-test':list()}
count = 0
for s in sparsity:
    for t_dim in target_dim:
        for s_rule in stop_rule:
            s_dict['sparsity'].append(s)
            s_dict['target_dim'].append(t_dim)
            s_dict['stop_rule'].append(s_rule)
            s_dict['zero-one-loss'].append(np.mean(scores_c1[count]))
            s_dict['zero-one-test'].append(np.mean(scores_c1_test[count]))
            count += 1  

#s_dict_c1 = s_dict
df = DataFrame(s_dict)

print('CART-DCTree-Breiman')
display(df)
display(np.mean(s_dict['zero-one-loss']))
display(np.mean(s_dict['zero-one-test']))

### My observations from preliminary experiments:
- Forest type estimators seem to perform uniformly worse than single-tree based estimators on sonar data
- Trees with adaptive pruning strategy (inspired by Kpotufe's paper) seems to perform significantly better than others, though it takes a lot more time 
- But combining forest with adaptive pruning doesn't help; it achieves similar bad performance as forest with naive stopping criteria

#### Some notes for future extensions
- Forest class: need to finish implementing the preprocessing on tree (ho's rotation forest) and forest level 
- Tree and Forest class: Need to add the option of multi-class prediction

In [11]:
### Test: twomeans projection (uniform sampling) + median split 
sample_rule_list = ['uniform', 'D2',]
target_dim_list = [1, 5, 10]
stop_rule_list = ['naive', 'cell_size']
scores_c2 = list()
scores_c2_test = list()
params_c2 = list()

##

for target_dim in target_dim_list:
    for stop_rule in stop_rule_list:
        for sample_rule in sample_rule_list:
            proj_design={'name':'projmat',
                         'params':{'name':'twomeans_proj','target_dim':target_dim}}
            split_design={'name':'median_perturb'} 
            stop_design={'name':stop_rule}
            if stop_rule == 'cell_size':
                ddiam, _, _ = data_diameter(data_tr)
                stop_design['params'] = {'diameter':ddiam, 
                                         'max_level':2*np.log(data_tr.shape[0])}
            if split_design['name'] == 'median_perturb':
                ddiam,_,_ = data_diameter(data_tr)
                split_design['params'] = {'diameter':ddiam}
            kwargs = {'proj_design':proj_design, 'split_design':split_design, 
                      'stop_design':stop_design}
            scores_c2.append(cross_valid_eval(data_tr, labels_tr, 5, 
                                              zero_one_loss, flex_binary_trees, 
                                              need_ind=True, **kwargs))
#scores_c1_test.append(cross_valid_eval(data_tt, labels_tt, 5, zero_one_loss, flex_binary_trees, 
#                                              need_ind=True, **kwargs))
#params_c2.append([s, t_dim, s_rule])

Evaluating the 1-th fold


array([False, False, False,  True,  True,  True,  True, False,  True,
        True, False,  True, False, False, False, False,  True,  True,
        True, False,  True, False,  True,  True,  True, False,  True,
       False, False, False,  True, False, False, False,  True,  True,
       False,  True, False,  True,  True,  True, False,  True, False,
       False, False, False,  True,  True, False,  True, False,  True,
       False, False, False,  True,  True, False,  True,  True, False,
       False, False, False, False,  True, False,  True, False,  True,
       False,  True,  True,  True,  True, False, False,  True, False,
        True,  True], dtype=bool)

array([ True,  True,  True, False, False, False, False,  True, False,
       False,  True, False,  True,  True,  True,  True, False, False,
       False,  True, False,  True, False, False, False,  True, False,
        True,  True,  True, False,  True,  True,  True, False, False,
        True, False,  True, False, False, False,  True, False,  True,
        True,  True,  True, False, False,  True, False,  True, False,
        True,  True,  True, False, False,  True, False, False,  True,
        True,  True,  True,  True, False,  True, False,  True, False,
        True, False, False, False, False,  True,  True, False,  True,
       False, False], dtype=bool)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True, False, False,  True], dtype=bool)

...

array([ True, False,  True,  True, False,  True, False, False, False,  True], dtype=bool)

array([ 0,  2,  3, 11, 14, 19, 28, 30, 34, 81])

array([False, False, False,  True, False], dtype=bool)

array([ True,  True,  True, False,  True], dtype=bool)

array([ 2, 14, 28, 30, 34])

In [12]:
s_dict = {'sampling':list(), 'target_dim':list(), 'stop_rule': list(), 'zero-one-loss':list()}
count = 0
for s in sample_rule_list:
    for t_dim in target_dim_list:
        for s_rule in stop_rule_list:
            s_dict['sampling'].append(s)
            s_dict['target_dim'].append(t_dim)
            s_dict['stop_rule'].append(s_rule)
            s_dict['zero-one-loss'].append(np.mean(scores_c2[count]))
            count += 1  

#s_dict_c1 = s_dict
df = DataFrame(s_dict)

print('Two-means-projection')
display(df)
display(np.mean(s_dict['zero-one-loss'][:5]))
display(np.mean(s_dict['zero-one-loss'][5:]))

Two-means-projection


,sampling,stop_rule,target_dim,zero-one-loss
0,uniform,naive,1,0.288095
1,uniform,cell_size,1,0.289524
2,uniform,naive,5,0.395238
3,uniform,cell_size,5,0.394286
4,uniform,naive,10,0.278095
5,uniform,cell_size,10,0.309048
6,D2,naive,1,0.394286
7,D2,cell_size,1,0.318095
8,D2,naive,5,0.297619
9,D2,cell_size,5,0.220952


0.32904761904761903

0.32027210884353741

In [36]:
####----- cluster based split
sample_rule_list = ['uniform','D2']
#sample_rule_list = ['D2']
target_dim_list = [1, 5, 10]
stop_rule_list = ['naive', 'cell_size'] 
scores_c2_1 = list()
scores_c2_1_test = list()
params_c2_1 = list()

##
for stop_rule in stop_rule_list:
    for sample_rule in sample_rule_list:
        proj_design={'name':'full'}
        split_design={'name':'cluster_based','params':{'sample_method':sample_rule}} 
        stop_design={'name':stop_rule}
        if stop_rule == 'cell_size':
            ddiam, _, _ = data_diameter(data_tr)
            stop_design['params'] = {'diameter':ddiam, 'max_level':2*np.log(data_tr.shape[0])}
        kwargs = {'proj_design':proj_design, 'split_design':split_design,'stop_design':stop_design}
        scores_c2_1.append(cross_valid_eval(data_tr, labels_tr, 5, 
                                            zero_one_loss, flex_binary_trees, 
                                            need_ind=True, **kwargs))

Evaluating the 1-th fold
Evaluating the 2-th fold
Evaluating the 3-th fold
Evaluating the 4-th fold
Evaluating the 5-th fold
Evaluating the 1-th fold
Evaluating the 2-th fold
Evaluating the 3-th fold
Evaluating the 4-th fold
Evaluating the 5-th fold
Evaluating the 1-th fold
Evaluating the 2-th fold
Evaluating the 3-th fold
Evaluating the 4-th fold
Evaluating the 5-th fold
Evaluating the 1-th fold
Evaluating the 2-th fold
Evaluating the 3-th fold
Evaluating the 4-th fold
Evaluating the 5-th fold


In [38]:
s_dict = {'sampling':list(),'stop_rule': list(), 'zero-one-loss':list()}
count = 0
for s in sample_rule_list:
    for s_rule in stop_rule_list:
        s_dict['sampling'].append(s)
        s_dict['stop_rule'].append(s_rule)
        s_dict['zero-one-loss'].append(np.mean(scores_c2_1[count]))
        count += 1  

#s_dict_c1 = s_dict
df = DataFrame(s_dict)

print('Two-means-split')
display(df)
display(np.mean(s_dict['zero-one-loss'][:2]))
display(np.mean(s_dict['zero-one-loss'][2:]))

Two-means-split


,sampling,stop_rule,zero-one-loss
0,uniform,naive,0.470952
1,uniform,cell_size,0.500952
2,D2,naive,0.450952
3,D2,cell_size,0.472381


0.48595238095238091

0.46166666666666661

In [24]:
## To be done
## 1. debugging: what is going on here
## 2. compare different variants of comparison-tree (as well as with other tree algs), and performance with RF
## 3. try algorithms on Gaussians (with low intrinsic dim), DL model
## 4. Compare single-tree (each tree grows to singleton) and forest performance 